Title: Video Games Analysis Throughout the world

In [199]:
#importing all necessary libraries
import pandas as pd
import streamlit as st
import plotly.express as px
from scipy.stats import ttest_ind


In [2]:
#loading the dataset in the panda data frame
df = pd.read_csv('../games.csv')

In [3]:
#viewing the Data
df.head()

,Name,Platform,Year_of_Release,Genre,NA_sales,EU_sales,JP_sales,Other_sales,Critic_Score,User_Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
#having an overview look over the data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16715 entries, 0 to 16714
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16713 non-null  object 
 1   Platform         16715 non-null  object 
 2   Year_of_Release  16446 non-null  float64
 3   Genre            16713 non-null  object 
 4   NA_sales         16715 non-null  float64
 5   EU_sales         16715 non-null  float64
 6   JP_sales         16715 non-null  float64
 7   Other_sales      16715 non-null  float64
 8   Critic_Score     8137 non-null   float64
 9   User_Score       10014 non-null  object 
 10  Rating           9949 non-null   object 
dtypes: float64(6), object(5)
memory usage: 1.4+ MB


In [118]:
# Changing all columns to lowercase
df.columns = df.columns.str.lower()
df.sample(5)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales_per_game,num_of_games_per_diff_years,total_sales
9484,Persona 4: Arena,X360,2012,Fighting,0.08,0.01,0.03,0.01,83,7.3,T,0.85,NaN,0.13
2973,Minecraft,WiiU,2016,Misc,0.28,0.17,0.18,0.04,70,tbd,E,24.16,NaN,0.67
2385,Metal Gear Ac!d,PSP,2004,Strategy,0.32,0.26,0.12,0.17,70,tbd,E,0.87,NaN,0.87
11230,Theme Hospital,PS,1998,Strategy,0.03,0.02,0.03,0.01,70,tbd,E,2.49,NaN,0.09
8469,Pursuit Force,PSP,2005,Racing,0.14,0.01,0.00,0.01,75,7.8,T,0.16,NaN,0.16


In [35]:
# inspecting data 
# checking for missing values
df.isna().sum()

name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

In [36]:
# filling missing values and changing the columns data type with contingent approach

# filling in the missing value of year_of_release columns then change its data type at the same time
#the reason why I am changing this data type to int is to make the data more reliable for analysis and it also uses less memory, plus Integer operations (e.g., comparisons, filtering by year) become easier and more efficient.
# an it also Helps avoid errors that may occur due to NaN or incorrect data types in future calculations.
df['year_of_release'] = df['year_of_release'].fillna(0).astype(int)
# filling missing value on genre according to its datatype
# with the help of the lambda anonymous function here i try to fill the missing value with the mode or most frequent value and if it does not exist i fill it in with unknown since its an object data type 
df['genre'] = df['genre'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
# filling in missing value of name column according to it's data type
# i use the same procedure just like the genre column since both columns are of object data types
df['name'] = df['name'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
# filling missing values for critic score and then change its data type to int
# and the reason why I am changing this data type is make it more consistent and easier to use just like year of release column
#fill the missing value with the mode or most frequent value and if it does not exist i fill it in with 0 since its an int data type 
df['critic_score'] = df['critic_score'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 0)).astype(int)
# filling missing value for user score accordingly
df['user_score'] = df['user_score'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
# filling missing value for rating column accordingly
df['rating'] = df['rating'].transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'unknown'))
df.isna().sum()



name               0
platform           0
year_of_release    0
genre              0
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64

Why do you think the values are missing? Give possible reasons.
1 of the reasons is because data was collected all across the regions and it is really hard to have perfect data
in different areas of the world
2. company in different regions might not have the same tools to collect the data
3. Data entry errors
4.innaplicable data
5. Data losss or optional field some field might not be mandatory for the respondents to respond to so might ended up skipping them


Dealing with TBD(to be determined):
I will Create a separate indicator column to flag rows with TBD, allowing me to revisit them when the data becomes available.

In [39]:
# checking for duplicates
df.duplicated().sum()
# no duplicates in this dataset

np.int64(0)

In [64]:
#total sales of each games in all region
df['total_sales_per_game'] = (df['na_sales'] + df['eu_sales'] + df['jp_sales'] + df['other_sales']).groupby(df['name']).transform('sum')
df.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales_per_game
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76,8,E,82.54
1,Super Mario Bros.,NES,1985,Platform,29.08,3.58,6.81,0.77,70,tbd,E,45.31
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80,8,E,32.77
4,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,11.27,8.89,10.22,1.00,70,tbd,E,31.38


Step3: Analyzing the Data

In [86]:
#looking at how many games were release in the different years
num_of_games_per_diff_years = df.groupby(df['year_of_release'])['name'].count()
num_of_games_per_diff_years
# as the year increases the number of games released increases as well

year_of_release
0        269
1980       9
1981      46
1982      36
1983      17
1984      14
1985      14
1986      21
1987      16
1988      15
1989      17
1990      16
1991      41
1992      43
1993      62
1994     121
1995     219
1996     263
1997     289
1998     379
1999     338
2000     350
2001     482
2002     829
2003     775
2004     762
2005     939
2006    1006
2007    1197
2008    1427
2009    1426
2010    1255
2011    1136
2012     653
2013     544
2014     581
2015     606
2016     502
Name: name, dtype: int64

In [93]:
#Distribution of platforms with the greatest total sales
top_platforms = df.groupby('platform')['total_sales_per_game'].sum().nlargest(3)
filtered_df = df[df['platform'].isin(top_platforms.index)]

sales_per_year = filtered_df.groupby(['year_of_release', 'platform'])['total_sales_per_game'].sum().reset_index()

fig = px.line(sales_per_year, x='year_of_release', y='total_sales_per_game', color='platform',
              title='Total Sales Distribution by Year for Top Platforms')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'platform=PS2<br>year_of_release=%{x}<br>total_sales_per_game=%{y}<extra></extra>',
              'legendgroup': 'PS2',
              'line': {'color': '#000001', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'PS2',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([   0, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
                          2011]),
              'xaxis': 'x',
              'y': array([ 36.16,  48.53, 208.18, 269.47, 270.03, 296.06, 252.83, 188.88, 196.4 ,
                          224.83, 120.14,  46.87,  20.17]),
              'yaxis': 'y'},
             {'hovertemplate': 'platform=PS3<br>year_of_release=%{x}<br>total_sales_per_game=%{y}<extra></extra>',
              'legendgroup': 'PS3',
              'line': {'color': '#000002', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'PS3',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([   0, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]),
              'xaxis': 'x',
              'y': array([ 30.31,  71.79, 223.35, 333.9 , 303.56, 334.07, 361.29, 254.8 , 306.63,
                          209.13, 113.91,  32.4 ]),
              'yaxis': 'y'},
             {'hovertemplate': ('platform=X360<br>year_of_relea' ... 's_per_game=%{y}<extra></extra>'),
              'legendgroup': 'X360',
              'line': {'color': '#000003', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'X360',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([   0, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
                          2016]),
              'xaxis': 'x',
              'y': array([ 37.26,  32.39, 146.07, 250.29, 353.88, 299.39, 361.21, 342.46, 248.44,
                          287.02, 178.77, 106.5 ,  25.19]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'platform'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Total Sales Distribution by Year for Top Platforms'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'year_of_release'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'total_sales_per_game'}}}
})

Q: Look at how sales varied from platform to platform. Choose the platforms with the greatest total sales and build a distribution based on data for each year. Find platforms that used to be popular but now have zero sales. How long does it generally take for new platforms to appear and old ones to fade?

In [94]:
# 1: group sales by platform
df['total_sales'] = df[['na_sales', 'eu_sales', 'jp_sales', 'other_sales']].sum(axis=1)
total_sales_per_platform = df.groupby('platform')['total_sales'].sum().sort_values(ascending=False)

In [95]:
#2: identifying top saling platform
top_platforms = total_sales_per_platform.head(5)
top_platforms

platform
PS2     1255.77
X360     971.42
PS3      939.65
Wii      907.51
DS       806.12
Name: total_sales, dtype: float64

In [96]:
#3: plot sales distribution for top platforms per year
top_platform_data = df[df['platform'].isin(top_platforms.index)]
fig = px.line(top_platform_data, x='year_of_release', y='total_sales', color='platform',
              title='Sales Distribution for Top Platforms by Year')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'platform=Wii<br>year_of_release=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': 'Wii',
              'line': {'color': '#000001', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'Wii',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([2006, 2008, 2009, ..., 2007, 2008, 2008]),
              'xaxis': 'x',
              'y': array([8.254e+01, 3.552e+01, 3.277e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'},
             {'hovertemplate': 'platform=DS<br>year_of_release=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': 'DS',
              'line': {'color': '#000002', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'DS',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([2006, 2005, 2005, ..., 2008, 2009, 2008]),
              'xaxis': 'x',
              'y': array([2.980e+01, 2.467e+01, 2.321e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'},
             {'hovertemplate': 'platform=X360<br>year_of_release=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': 'X360',
              'line': {'color': '#000003', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'X360',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([2010, 2013, 2011, ..., 2011, 2013, 2006]),
              'xaxis': 'x',
              'y': array([2.182e+01, 1.627e+01, 1.473e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'},
             {'hovertemplate': 'platform=PS3<br>year_of_release=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': 'PS3',
              'line': {'color': '#000004', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'PS3',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([2013, 2012, 2011, ..., 2008, 2016, 2016]),
              'xaxis': 'x',
              'y': array([2.105e+01, 1.379e+01, 1.333e+01, ..., 0.000e+00, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'},
             {'hovertemplate': 'platform=PS2<br>year_of_release=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': 'PS2',
              'line': {'color': '#000005', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'PS2',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([2004, 2002, 2001, ..., 2008, 2008, 2008]),
              'xaxis': 'x',
              'y': array([2.081e+01, 1.615e+01, 1.497e+01, ..., 1.000e-02, 1.000e-02, 0.000e+00]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'platform'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Sales Distribution for Top Platforms by Year'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'year_of_release'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'total_sales'}}}
})

Q: Determine what period you should take data for. To do so, look at your answers to the previous questions. The data should allow you to build a model for 2017.

In [123]:
# based on the previous questions the best period we should take data for is from the year 2000
df_year = df[df['year_of_release']>=2000]
df_year.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,total_sales_per_game,num_of_games_per_diff_years,total_sales
0,Wii Sports,Wii,2006,Sports,41.36,28.96,3.77,8.45,76,8,E,82.54,269.0,82.54
2,Mario Kart Wii,Wii,2008,Racing,15.68,12.76,3.79,3.29,82,8.3,E,35.52,NaN,35.52
3,Wii Sports Resort,Wii,2009,Sports,15.61,10.93,3.28,2.95,80,8,E,32.77,NaN,32.77
6,New Super Mario Bros.,DS,2006,Platform,11.28,9.14,6.50,2.88,89,8.5,E,29.80,NaN,29.80
7,Wii Play,Wii,2006,Misc,13.96,9.18,2.93,2.84,58,6.6,E,28.91,NaN,28.91


Q: Which platforms are leading in sales? Which ones are growing or shrinking? Select several potentially profitable platforms.

In [125]:
# to determine this I would like to build a relationship scatter plot
scatter = px.scatter(df_year, x='platform', y='total_sales',title='Sales Vs Platform', labels={'platform':'Game Platform',
                                                                                          'total_sales':'Total Sales per Platform'},
                                                                                          color_discrete_sequence=['blue']
                                                                                          )
#display figure
scatter

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Game Platform=%{x}<br>Total Sales per Platform=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'blue', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array(['Wii', 'Wii', 'Wii', ..., 'PSV', 'GBA', 'PSV'], dtype=object),
              'xaxis': 'x',
              'y': array([8.254e+01, 3.552e+01, 3.277e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Sales Vs Platform'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Game Platform'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Total Sales per Platform'}}}
})

The platforms that leading the gaming industry are: Wii, DS, X360, PS3, PS2, GBA PS4 3DS.
the Platforms that are shrinking are : XB ,PC, PSP, XONE WIIU GC, GB, PS , N64, PSV, DC, WS


Q: Build a box plot for the global sales of all games, broken down by platform. Are the differences in sales significant? What about average sales on various platforms? Describe your findings.

In [128]:
# box plot
fig = px.box(df, x='platform', y='total_sales', title='Global Sales Distribution by Platform',color_discrete_sequence=['green'])
#dislpaying the plot
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'platform=%{x}<br>total_sales=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'green'},
              'name': '',
              'notched': False,
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'box',
              'x': array(['Wii', 'NES', 'Wii', ..., 'PSV', 'GBA', 'PSV'], dtype=object),
              'x0': ' ',
              'xaxis': 'x',
              'y': array([8.254e+01, 4.024e+01, 3.552e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'y0': ' ',
              'yaxis': 'y'}],
    'layout': {'boxmode': 'group',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Global Sales Distribution by Platform'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'platform'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'total_sales'}}}
})

Q: Take a look at how user and professional reviews affect sales for one popular platform (you choose). Build a scatter plot and calculate the correlation between reviews and sales. Draw conclusions.

In [144]:
#grouping the new df by a popular platform
pop_df = df[df['platform'] == 'Wii']
# buildeing a scatter plot
scatter = px.scatter(pop_df, x='critic_score', y='total_sales', title='Review Vs Sales', labels={
   'total_sales':'Sales', 'critic_score': 'Reviews'}, color_discrete_sequence=['green'] )
scatter


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'Reviews=%{x}<br>Sales=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'green', 'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([76, 82, 80, ..., 70, 70, 70]),
              'xaxis': 'x',
              'y': array([8.254e+01, 3.552e+01, 3.277e+01, ..., 1.000e-02, 1.000e-02, 1.000e-02]),
              'yaxis': 'y'}],
    'layout': {'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Review Vs Sales'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Reviews'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Sales'}}}
})

my conclusion based on this graph is that the popular the platform the more reviews it has 

Q: Keeping your conclusions in mind, compare the sales of the same games on other platforms.

In [165]:
# filtering df for other platorms
other_df = df[df['name'].isin(pop_df['name'])]
other_df = other_df[other_df['platform'] != 'Wii']
other_df = other_df[other_df['year_of_release']>= 2006]
other_df = other_df[other_df['total_sales_per_game'] > 10]
#building a histogram to compare the sales of other games
bar_graph = px.bar(other_df,x='name',y='total_sales_per_game',title='Sales Distribution in games',color_discrete_sequence=['blue'])
# Adjusting bar width and space between bars
bar_graph.update_traces(marker_line_width=1.5)
bar_graph.update_layout(xaxis={'categoryorder':'total descending'}, bargap=0.3)

bar_graph
#other_df.head()


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'name=%{x}<br>total_sales_per_game=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'blue', 'line': {'width': 1.5}, 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Call of Duty: Modern Warfare 3', 'Call of Duty: Black Ops',
                          'Call of Duty: Modern Warfare 3', 'Call of Duty: Black Ops',
                          'FIFA Soccer 13', 'The Sims 3', 'Call of Duty: World at War', 'FIFA 12',
                          'FIFA 14', 'FIFA 15', 'Call of Duty: World at War', 'FIFA Soccer 13',
                          'Mario & Sonic at the Olympic Games', 'FIFA Soccer 11',
                          'Guitar Hero III: Legends of Rock', 'LEGO Star Wars: The Complete Saga',
                          'Guitar Hero III: Legends of Rock', 'Madden NFL 07', 'FIFA 15',
                          'FIFA 14', 'FIFA 12', 'LEGO Indiana Jones: The Original Adventures',
                          'FIFA Soccer 10', 'FIFA Soccer 11', 'LEGO Batman: The Videogame',
                          'LEGO Batman: The Videogame', 'FIFA 14', 'FIFA 15',
                          'Star Wars: The Force Unleashed', 'Guitar Hero: World Tour',
                          'FIFA Soccer 10', 'LEGO Star Wars: The Complete Saga',
                          'LEGO Star Wars: The Complete Saga', 'The Sims 3',
                          'Guitar Hero III: Legends of Rock', 'Star Wars: The Force Unleashed',
                          'LEGO Indiana Jones: The Original Adventures', 'FIFA 15',
                          'Just Dance 3', 'The Sims 3', 'Guitar Hero: World Tour',
                          'Guitar Hero: World Tour', 'Madden NFL 07',
                          'Call of Duty: Modern Warfare 3', 'FIFA Soccer 10',
                          'LEGO Indiana Jones: The Original Adventures',
                          'LEGO Batman: The Videogame', 'LEGO Batman: The Videogame',
                          'Star Wars: The Force Unleashed', 'FIFA Soccer 11',
                          'LEGO Indiana Jones: The Original Adventures', 'FIFA 14',
                          'FIFA Soccer 10', 'Star Wars: The Force Unleashed',
                          'LEGO Indiana Jones: The Original Adventures', 'Madden NFL 07',
                          'Call of Duty: World at War', 'Star Wars: The Force Unleashed',
                          'FIFA Soccer 11', 'The Sims 3', 'FIFA Soccer 11', 'The Sims 3',
                          'Madden NFL 07', 'Just Dance 3', 'FIFA 15', 'Call of Duty: Black Ops',
                          'FIFA Soccer 10', 'Madden NFL 07', 'FIFA 12', 'FIFA Soccer 13',
                          'FIFA 12', 'FIFA 15', 'FIFA Soccer 13', 'FIFA 14', 'FIFA 14',
                          'FIFA Soccer 13', 'FIFA 12', 'FIFA Soccer 13', 'FIFA Soccer 13',
                          'FIFA 15', 'FIFA 14', 'Madden NFL 07', 'FIFA 14',
                          'Call of Duty: World at War', 'FIFA Soccer 11', 'FIFA 12',
                          'LEGO Batman: The Videogame', 'FIFA Soccer 10',
                          'LEGO Indiana Jones: The Original Adventures',
                          'Guitar Hero: World Tour', 'Madden NFL 07',
                          'Guitar Hero III: Legends of Rock'], dtype=object),
              'xaxis': 'x',
              'y': array([30.6 , 30.82, 30.6 , 30.82, 15.97, 15.06, 15.78, 13.03, 16.46, 17.37,
                          15.78, 15.97, 13.09, 12.49, 16.37, 15.25, 16.37, 10.06, 17.37, 16.46,
                          13.03, 11.9 , 10.69, 12.49, 13.4 , 13.4 , 16.46, 17.37, 10.02, 10.04,
                          10.69, 15.25, 15.25, 15.06, 16.37, 10.02, 11.9 , 17.37, 12.76, 15.06,
                          10.04, 10.04, 10.06, 30.6 , 10.69, 11.9 , 13.4 , 13.

In conclusion sport related games on other platforms made more sales than non related sport games 

Q: Take a look at the general distribution of games by genre. What can we say about the most profitable genres? Can you generalize about genres with high and low sales?

In [167]:
# filtering the df by profits
profit_df = df[df['year_of_release'] > 2000]
profit_df = profit_df[profit_df['total_sales_per_game']> 10]
# building a barchart
pro_chart = px.bar(profit_df,x='genre', y='total_sales_per_game',title='Most Profitable genre games', color_discrete_sequence=['green'])
pro_chart

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'genre=%{x}<br>total_sales_per_game=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'green', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Sports', 'Racing', 'Sports', ..., 'Sports', 'Misc', 'Action'],
                         dtype=object),
              'xaxis': 'x',
              'y': array([82.54, 35.52, 32.77, ..., 10.06, 16.37, 13.11]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Most Profitable genre games'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'genre'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'total_sales_per_game'}}}
})

In conclusion based on this graph the most profitable genres are: shooter, action, sports 

Q: Step 4. Create a user profile for each region

For each region (NA, EU, JP), determine:

The top five platforms. Describe variations in their market shares from region to region.

In [169]:
#filtering data by region
#NA Region
na_sales_df = df[['platform','name','na_sales']].groupby('platform').sum().sort_values(by='na_sales',ascending=False).head(5)
na_sales_df

,name,na_sales
platform,,
X360,Kinect Adventures!Grand Theft Auto VCall of Du...,602.47
PS2,Grand Theft Auto: San AndreasGrand Theft Auto:...,583.84
Wii,Wii SportsMario Kart WiiWii Sports ResortWii P...,496.90
PS3,Grand Theft Auto VCall of Duty: Black Ops IICa...,393.49
DS,New Super Mario Bros.NintendogsMario Kart DSBr...,382.40


In [171]:
# Eu region
eu_sales_df = df[['platform','name','eu_sales']].groupby('platform').sum().sort_values(by='eu_sales',ascending=False).head(5)
eu_sales_df.head()

,name,eu_sales
platform,,
PS2,Grand Theft Auto: San AndreasGrand Theft Auto:...,339.29
PS3,Grand Theft Auto VCall of Duty: Black Ops IICa...,330.29
X360,Kinect Adventures!Grand Theft Auto VCall of Du...,270.76
Wii,Wii SportsMario Kart WiiWii Sports ResortWii P...,262.21
PS,Gran TurismoFinal Fantasy VIIGran Turismo 2Fin...,213.61


In [172]:
#JP Region
jp_sales_df = df[['platform','name','jp_sales']].groupby('platform').sum().sort_values(by='jp_sales',ascending=False).head(5)
jp_sales_df

,name,jp_sales
platform,,
DS,New Super Mario Bros.NintendogsMario Kart DSBr...,175.57
PS,Gran TurismoFinal Fantasy VIIGran Turismo 2Fin...,139.82
PS2,Grand Theft Auto: San AndreasGrand Theft Auto:...,139.20
SNES,Super Mario WorldSuper Mario All-StarsDonkey K...,116.55
3DS,Pokemon X/Pokemon YMario Kart 7Pokemon Omega R...,100.67


the North american region has the bigest market shares in all the top 5 platforms followed by eu region then japan region

Q: The top five genres. Explain the difference.

In [174]:
# top five genres in each region
# Na_region
na_genre_df = df[['name','genre','na_sales']].groupby('genre').sum().sort_values(by='na_sales',ascending=False).head(5)
na_genre_df


,name,na_sales
genre,,
Action,Grand Theft Auto VGrand Theft Auto: San Andrea...,880.79
Sports,Wii SportsWii Sports ResortWii FitWii Fit Plus...,684.43
Shooter,Duck HuntCall of Duty: Modern Warfare 3Call of...,592.24
Platform,Super Mario Bros.New Super Mario Bros.New Supe...,445.50
Misc,Wii PlayKinect Adventures!Brain Age: Train You...,407.27


In [175]:
# top five genres in each region
# EU_region
eu_genre_df = df[['genre','name','eu_sales']].groupby('genre').sum().sort_values(by='eu_sales',ascending=False).head(5)
eu_genre_df

,name,eu_sales
genre,,
Action,Grand Theft Auto VGrand Theft Auto: San Andrea...,519.66
Sports,Wii SportsWii Sports ResortWii FitWii Fit Plus...,376.79
Shooter,Duck HuntCall of Duty: Modern Warfare 3Call of...,317.34
Racing,Mario Kart WiiMario Kart DSGran Turismo 3: A-S...,236.51
Misc,Wii PlayKinect Adventures!Brain Age: Train You...,212.74


In [176]:
# top five genres in each region
# JP Region
jp_genre_df = df[['genre','name','jp_sales']].groupby('genre').sum().sort_values(by='jp_sales',ascending=False).head(5)
jp_genre_df

,name,jp_sales
genre,,
Role-Playing,Pokemon Red/Pokemon BluePokemon Gold/Pokemon S...,355.41
Action,Grand Theft Auto VGrand Theft Auto: San Andrea...,161.46
Sports,Wii SportsWii Sports ResortWii FitWii Fit Plus...,135.54
Platform,Super Mario Bros.New Super Mario Bros.New Supe...,130.83
Misc,Wii PlayKinect Adventures!Brain Age: Train You...,108.11


in all regions the same genres top all the sales and like the platforms the north american region perfoms better in sales followed by the eu region and then japan region


Q: Do ESRB ratings affect sales in individual regions?

In [181]:
# grouping by ratins
esrb_by_region = df.groupby('rating')[['na_sales','eu_sales','jp_sales']].sum()
# building a grapgh
esrb_graph = px.bar(esrb_by_region, barmode='group',title='ESRB RATING VS SALES BY REGION',color_discrete_sequence=['green'])
esrb_graph

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'variable=na_sales<br>rating=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'na_sales',
              'marker': {'color': 'green', 'pattern': {'shape': ''}},
              'name': 'na_sales',
              'offsetgroup': 'na_sales',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T'], dtype=object),
              'xaxis': 'x',
              'y': array([1.26000e+00, 2.53545e+03, 3.53320e+02, 1.53000e+00, 2.56000e+00,
                          7.48480e+02, 0.00000e+00, 7.59750e+02]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=eu_sales<br>rating=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'eu_sales',
              'marker': {'color': 'green', 'pattern': {'shape': ''}},
              'name': 'eu_sales',
              'offsetgroup': 'eu_sales',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T'], dtype=object),
              'xaxis': 'x',
              'y': array([6.10000e-01, 1.32408e+03, 1.88520e+02, 1.10000e-01, 2.70000e-01,
                          4.83970e+02, 8.00000e-02, 4.27030e+02]),
              'yaxis': 'y'},
             {'alignmentgroup': 'True',
              'hovertemplate': 'variable=jp_sales<br>rating=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'jp_sales',
              'marker': {'color': 'green', 'pattern': {'shape': ''}},
              'name': 'jp_sales',
              'offsetgroup': 'jp_sales',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T'], dtype=object),
              'xaxis': 'x',
              'y': array([   0.  , 1040.07,   40.2 ,    0.  ,    1.46,   64.24,    0.  ,  151.4 ]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'group',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'ESRB RATING VS SALES BY REGION'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'rating'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'value'}}}
})

The E rating top the sales in all regions so in conclusions ratings do affect sales based on this study

Q: Step 5. Test the following hypotheses:

—Average user ratings of the Xbox One and PC platforms are the same. 

—Average user ratings for the Action and Sports genres are different.

Set the alpha threshold value yourself.

Hypothesis 1:
Average user ratings of Xbox One and PC platforms are the same.

1. Null Hypothesis (H₀): The mean user ratings of Xbox One = PC.
2. Alternative Hypothesis (H₁): The mean user ratings of Xbox One different from PC.

Hypothesis 2:
Average user ratings for Action and Sports genres are different.

1. Null Hypothesis (H₀): The mean user ratings for Action = Sports.
2. Alternative Hypothesis (H₁): The mean user ratings for Action different Sports.


In [210]:
# setting alpha
alpha = 0.05
#filtering data
#1: xbox data
xbox_data = df[df['platform'] == 'XOne']
#2: pc data
pc_data = df[df['platform'] == 'PC']
#3: action data
action_data = df[df['genre']== 'Action']
#4 : sports data
sports_data = df[df['genre'] == 'Sports']
# Xbox One vs PC
# to make this work i replace all strings values in the columns to a numeric value then converted the data type to float to be able to do the math
t_stat_xbox_pc, p_value_xbox_pc = ttest_ind(xbox_data['user_score'].replace('tbd', 0).astype(float).dropna(), pc_data['user_score'].replace('tbd', 0).astype(float).dropna())


# Action vs Sports
t_stat_action_sports, p_value_action_sports = ttest_ind(action_data['user_score'].replace('tbd',0).astype(float).dropna(), sports_data['user_score'].replace('tbd',0).astype(float).dropna())


In [211]:
# Analyzing my P Value
if p_value_xbox_pc < alpha:
    print("Reject null hypothesis for Xbox One vs PC")
else:
    print("Fail to reject null hypothesis for Xbox One vs PC")

if p_value_action_sports < alpha:
    print("Reject null hypothesis for Action vs Sports")
else:
    print("Fail to reject null hypothesis for Action vs Sports")



Reject null hypothesis for Xbox One vs PC
Reject null hypothesis for Action vs Sports
